In [44]:
import pandas as pd
import numpy as np

In [2]:
stage1 = pd.read_csv('stage1.csv')
stage1.head()

,Rnk,GC,GC-Time,BIB,Rider,Age,Team,UCI,Pnt,Time,sprint_pts,kom_pts,stage_no,sprints,koms
0,1,1.0,+0:00,135,kristoff alexander,33,UAE-Team Emirates,120.0,100.0,3:46:233:46:23,59,0,1,2,2
1,2,2.0,+0:04,105,pedersen mads,24,Trek - Segafredo,50.0,70.0,",,0:00",30,0,1,2,2
2,3,3.0,+0:06,203,bol cees,25,Team Sunweb,25.0,50.0,",,0:00",20,0,1,2,2
3,4,4.0,+0:10,43,bennett sam,29,Deceuninck - Quick Step,15.0,40.0,",,0:00",28,0,1,2,2
4,5,5.0,+0:10,21,sagan peter,30,BORA - hansgrohe,5.0,32.0,",,0:00",29,0,1,2,2


In [127]:
riders_profile = stage1.loc[:,['Rider','Age','Team']]
riders_profile.head()

,Rider,Age,Team
0,kristoff alexander,33,UAE-Team Emirates
1,pedersen mads,24,Trek - Segafredo
2,bol cees,25,Team Sunweb
3,bennett sam,29,Deceuninck - Quick Step
4,sagan peter,30,BORA - hansgrohe


In [4]:
import requests

# PCS individual ranking:
Summation of PCS points over a 12-month + 2 weeks overlap period. Races are counted once. After the finish of a stage, the corresponding stage of the previous edition is excluded. 


In [128]:
for page in np.arange(0,2000,200):
    url = f'https://www.procyclingstats.com/rankings.php?id=59874&nation=&team=&page={page}&prev_id=51687&younger=&older=&limit=200&filter=Filter&morefilters=1'
    pcs_url = requests.get(url)
    print(page,pcs_url)
    body = pcs_url.text
    pcs_individual_data_n = pd.read_html(body)
    if page>0:
        pcs_individual_data = pcs_individual_data.append(pd.DataFrame(pcs_individual_data_n[0]),ignore_index=True)
    else:
        pcs_individual_data = pd.DataFrame(pcs_individual_data_n[0])

0 <Response [200]>
200 <Response [200]>
400 <Response [200]>
600 <Response [200]>
800 <Response [200]>
1000 <Response [200]>
1200 <Response [200]>
1400 <Response [200]>
1600 <Response [200]>
1800 <Response [200]>


In [129]:
pcs_individual_data.dropna(inplace=True)

In [130]:
pcs_individual_data['Diff.'] = pcs_individual_data['Diff.'].apply(lambda x: x.replace(chr(9650),''))
pcs_individual_data['Diff.'] = pcs_individual_data['Diff.'].apply(lambda x: x.replace('-','0'))
pcs_individual_data['Diff.'] = pcs_individual_data['Diff.'].apply(lambda x: x.replace(chr(9660),'-'))

In [131]:
pcs_individual_data.drop(columns=['Unnamed: 6'],inplace=True)

In [37]:
def clean_pcs_diff(column):
    column = column.apply(lambda x: x.replace(chr(9650),''))
    column = column.apply(lambda x: x.replace('-','0'))
    column = column.apply(lambda x: x.replace(chr(9660),'-'))
    return column

In [132]:
import unidecode
pcs_individual_data.Rider = pcs_individual_data.Rider.apply(lambda x: unidecode.unidecode(x.lower()))

In [133]:
pcs_individual_data.Rider.isin(riders_profile.Rider).sum()

172

In [93]:
def keep_tdf_riders(data,tdf_riders):
    data = data[data.Rider.isin(tdf_riders.Rider)]
    return data

In [94]:
keep_tdf_riders(pcs_individual_data,riders_profile)

,Rnk,Prev.,Diff.,Rider,Team,Pnts
0,1,10.0,9,pogacar tadej,UAE-Team Emirates,1302
1,2,1.0,-1,roglic primoz,Team Jumbo-Visma,1281
5,6,7.0,1,van avermaet greg,CCC Team,1046
6,7,12.0,5,quintana nairo,Team Arkéa Samsic,993
8,9,6.0,-3,valverde alejandro,Movistar Team,917
...,...,...,...,...,...,...
1224,1225,330.0,-895,debusschere jens,B&B Hotels - Vital Concept p/b KTM,9
1255,1256,781.0,-475,sicard romain,Team Total Direct Energie,8
1343,1344,618.0,-726,cousin jerome,Team Total Direct Energie,6
1399,1400,487.0,-913,anacona winner,Team Arkéa Samsic,5


In [135]:
riders_profile = riders_profile.merge(keep_tdf_riders(pcs_individual_data,riders_profile),how='left',on='Rider',suffixes=('','_pcs_ind'))

In [136]:
riders_profile.drop(columns='Team_pcs_ind',inplace=True)
riders_profile.drop(columns='Prev.',inplace=True)

In [137]:
riders_profile = riders_profile.rename(columns={'Rnk':'Rnk_pcs_ind','Diff.':'Diff._pcs_ind','Pnts':'Pnts_pcs_ind'})

In [138]:
riders_profile

,Rider,Age,Team,Rnk_pcs_ind,Diff._pcs_ind,Pnts_pcs_ind
0,kristoff alexander,33,UAE-Team Emirates,28.0,-20,611.0
1,pedersen mads,24,Trek - Segafredo,26.0,72,625.0
2,bol cees,25,Team Sunweb,141.0,-39,237.0
3,bennett sam,29,Deceuninck - Quick Step,12.0,6,828.0
4,sagan peter,30,BORA - hansgrohe,19.0,-6,684.0
...,...,...,...,...,...,...
171,sivakov pavel,23,INEOS Grenadiers,57.0,-19,431.0
172,cras steff,24,Lotto Soudal,502.0,97,58.0
173,ledanois kevin,27,Team Arkéa Samsic,420.0,148,75.0
174,valls rafael,33,Bahrain - McLaren,807.0,-143,24.0


# PCS climbers ranking:
Summation of PCS points over a 12-month + 2 weeks overlap period. Races are counted once. After the finish of a stage, the corresponding stage of the previous edition is excluded. 

In [139]:
for page in np.arange(0,1000,200):
    url = f'https://www.procyclingstats.com/rankings.php?id=60972&nation=&team=&page={page}&prev_id=prev&younger=&older=&limit=200&filter=Filter&morefilters='
    pcs_url = requests.get(url)
    print(page,pcs_url)
    body = pcs_url.text
    pcs_climb_data_n = pd.read_html(body)
    if page>0:
        pcs_climb_data = pcs_climb_data.append(pd.DataFrame(pcs_climb_data_n[0]),ignore_index=True)
    else:
        pcs_climb_data = pd.DataFrame(pcs_climb_data_n[0])

0 <Response [200]>
200 <Response [200]>
400 <Response [200]>
600 <Response [200]>
800 <Response [200]>


In [140]:
pcs_climb_data.dropna(inplace=True)
pcs_climb_data['Diff.'] = pcs_climb_data['Diff.'].apply(lambda x: x.replace(chr(9650),''))
pcs_climb_data['Diff.'] = pcs_climb_data['Diff.'].apply(lambda x: x.replace('-','0'))
pcs_climb_data['Diff.'] = pcs_climb_data['Diff.'].apply(lambda x: x.replace(chr(9660),'-'))
pcs_climb_data

,Rnk,Prev.,Diff.,Rider,Team,Pnts,Unnamed: 6
0,1,5.0,4,Pogačar Tadej,UAE-Team Emirates,956,Point breakdownStandings per season
1,2,1.0,-1,Roglič Primož,Team Jumbo-Visma,819,Point breakdownStandings per season
2,3,12.0,9,Yates Adam,Mitchelton-Scott,560,Point breakdownStandings per season
3,4,56.0,52,Porte Richie,Trek - Segafredo,535,Point breakdownStandings per season
4,5,31.0,26,Martin Guillaume,"Cofidis, Solutions Crédits",500,Point breakdownStandings per season
...,...,...,...,...,...,...,...
862,863,1553.0,690,Yamamoto Masaki,Kinan Cycling Team,0,Point breakdownStandings per season
864,865,1460.0,595,Spreafico Matteo,Vini Zabù - KTM,0,Point breakdownStandings per season
865,866,1558.0,692,Nakamura Kaito,Utsunomiya Blitzen,0,Point breakdownStandings per season
866,867,715.0,-152,Deltombe Kevin,Sport Vlaanderen - Baloise,0,Point breakdownStandings per season


In [141]:
pcs_climb_data.Rider = pcs_climb_data.Rider.apply(lambda x: unidecode.unidecode(x.lower()))
pcs_climb_data.drop(columns=['Unnamed: 6'],inplace=True)

In [142]:
riders_profile = riders_profile.merge(keep_tdf_riders(pcs_climb_data,riders_profile),how='left',on='Rider',suffixes=('','_pcs_climb'))

In [143]:
riders_profile = riders_profile.rename(columns={'Rnk':'Rnk_climb_ind','Diff.':'Diff._climb_ind','Pnts':'Pnts_climb_ind'})
riders_profile.drop(columns=['Prev.','Team_pcs_climb'],inplace=True)

In [144]:
riders_profile.sort_values(by='Rnk_pcs_ind')

,Rider,Age,Team,Rnk_pcs_ind,Diff._pcs_ind,Pnts_pcs_ind,Rnk_climb_ind,Diff._climb_ind,Pnts_climb_ind
16,pogacar tadej,21,UAE-Team Emirates,1.0,9,1302.0,1.0,4,956.0
38,roglic primoz,30,Team Jumbo-Visma,2.0,-1,1281.0,2.0,-1,819.0
67,van avermaet greg,35,CCC Team,6.0,1,1046.0,31.0,-15,286.0
59,quintana nairo,30,Team Arkéa Samsic,7.0,5,993.0,18.0,-14,384.0
121,valverde alejandro,40,Movistar Team,9.0,-3,917.0,9.0,-7,476.0
...,...,...,...,...,...,...,...,...,...
153,niv guy,26,Israel Start-Up Nation,1458.0,-781,5.0,NaN,NaN,NaN
54,grellier fabien,25,Team Total Direct Energie,NaN,NaN,NaN,NaN,NaN,NaN
55,reza kevin,32,B&B Hotels - Vital Concept p/b KTM,NaN,NaN,NaN,NaN,NaN,NaN
73,perichon pierre-luc,33,"Cofidis, Solutions Crédits",NaN,NaN,NaN,NaN,NaN,NaN


# PCS sprints ranking:
Summation of PCS points over a 12-month + 2 weeks overlap period. Races are counted once. After the finish of a stage, the corresponding stage of the previous edition is excluded. 

In [145]:
for page in np.arange(0,1000,200):
    url = f'https://www.procyclingstats.com/rankings.php?id=60937&nation=&team=&page={page}&prev_id=prev&younger=&older=&limit=200&filter=Filter&morefilters='
    pcs_url = requests.get(url)
    print(page,pcs_url)
    body = pcs_url.text
    pcs_sprint_data_n = pd.read_html(body)
    if page>0:
        pcs_sprint_data = pcs_sprint_data.append(pd.DataFrame(pcs_sprint_data_n[0]),ignore_index=True)
    else:
        pcs_sprint_data = pd.DataFrame(pcs_sprint_data_n[0])

0 <Response [200]>
200 <Response [200]>
400 <Response [200]>
600 <Response [200]>
800 <Response [200]>


In [146]:
pcs_sprint_data.dropna(inplace=True)
pcs_sprint_data['Diff.'] = pcs_sprint_data['Diff.'].apply(lambda x: x.replace(chr(9650),''))
pcs_sprint_data['Diff.'] = pcs_sprint_data['Diff.'].apply(lambda x: x.replace('-','0'))
pcs_sprint_data['Diff.'] = pcs_sprint_data['Diff.'].apply(lambda x: x.replace(chr(9660),'-'))
pcs_sprint_data.Rider = pcs_sprint_data.Rider.apply(lambda x: unidecode.unidecode(x.lower()))
pcs_sprint_data.drop(columns=['Unnamed: 6'],inplace=True)
pcs_sprint_data

,Rnk,Prev.,Diff.,Rider,Team,Pnts
0,1,3.0,2,ackermann pascal,BORA - hansgrohe,857.0
1,2,4.0,2,bennett sam,Deceuninck - Quick Step,736.5
2,3,14.0,11,gaviria fernando,UAE-Team Emirates,633.5
3,4,18.0,14,demare arnaud,Groupama - FDJ,580.5
4,5,23.0,18,nizzolo giacomo,NTT Pro Cycling,535.5
...,...,...,...,...,...,...
982,983,679.0,-304,gazzoli michele,Team Colpack Ballan,9.0
984,985,881.0,-104,krizek matthias,Team Felbermayr - Simplon Wels,9.0
989,990,901.0,-89,antomarchi julien,Natura4Ever - Roubaix Lille Métropole,8.0
991,992,333.0,-659,jerman ziga,Equipe continentale Groupama-FDJ,8.0


In [147]:
riders_profile = riders_profile.merge(keep_tdf_riders(pcs_sprint_data,riders_profile),how='left',on='Rider',suffixes=('','_pcs_sprint'))

In [148]:
riders_profile = riders_profile.rename(columns={'Rnk':'Rnk_sprint_ind','Diff.':'Diff._sprint_ind','Pnts':'Pnts_sprint_ind'})
riders_profile.drop(columns=['Prev.','Team_pcs_sprint'],inplace=True)

In [149]:
riders_profile

,Rider,Age,Team,Rnk_pcs_ind,Diff._pcs_ind,Pnts_pcs_ind,Rnk_climb_ind,Diff._climb_ind,Pnts_climb_ind,Rnk_sprint_ind,Diff._sprint_ind,Pnts_sprint_ind
0,kristoff alexander,33,UAE-Team Emirates,28.0,-20,611.0,NaN,NaN,NaN,17.0,-6,335.5
1,pedersen mads,24,Trek - Segafredo,26.0,72,625.0,396.0,128,10.0,30.0,94,270.5
2,bol cees,25,Team Sunweb,141.0,-39,237.0,687.0,-365,2.0,37.0,18,227.5
3,bennett sam,29,Deceuninck - Quick Step,12.0,6,828.0,128.0,301,58.0,2.0,2,736.5
4,sagan peter,30,BORA - hansgrohe,19.0,-6,684.0,116.0,-73,68.0,8.0,1,428.0
...,...,...,...,...,...,...,...,...,...,...,...,...
171,sivakov pavel,23,INEOS Grenadiers,57.0,-19,431.0,45.0,5,225.0,NaN,NaN,NaN
172,cras steff,24,Lotto Soudal,502.0,97,58.0,225.0,169,24.0,NaN,NaN,NaN
173,ledanois kevin,27,Team Arkéa Samsic,420.0,148,75.0,NaN,NaN,NaN,343.0,86,43.0
174,valls rafael,33,Bahrain - McLaren,807.0,-143,24.0,354.0,151,12.0,NaN,NaN,NaN


### SAVED DATA WITH RANK,CLIMB,SPRINT FROM PCS

In [150]:
riders_profile.to_csv('riders_profile.csv')

# PCS GC ranking:
Summation of PCS points over a 12-month + 2 weeks overlap period. Races are counted once. After the finish of a stage, the corresponding stage of the previous edition is excluded. 

In [151]:
for page in np.arange(0,1000,200):
    url = f'https://www.procyclingstats.com/rankings.php?id=50936&nation=&team=&page={page}&prev_id=prev&younger=&older=&limit=200&filter=Filter&morefilters='
    pcs_url = requests.get(url)
    print(page,pcs_url)
    body = pcs_url.text
    pcs_gc_data_n = pd.read_html(body)
    if page>0:
        pcs_gc_data = pcs_gc_data.append(pd.DataFrame(pcs_gc_data_n[0]),ignore_index=True)
    else:
        pcs_gc_data = pd.DataFrame(pcs_gc_data_n[0])

0 <Response [200]>
200 <Response [200]>
400 <Response [200]>
600 <Response [200]>
800 <Response [200]>


In [152]:
pcs_gc_data.dropna(inplace=True)
pcs_gc_data['Diff.'] = pcs_gc_data['Diff.'].apply(lambda x: x.replace(chr(9650),''))
pcs_gc_data['Diff.'] = pcs_gc_data['Diff.'].apply(lambda x: x.replace('-','0'))
pcs_gc_data['Diff.'] = pcs_gc_data['Diff.'].apply(lambda x: x.replace(chr(9660),'-'))
pcs_gc_data.Rider = pcs_gc_data.Rider.apply(lambda x: unidecode.unidecode(x.lower()))
pcs_gc_data.drop(columns=['Unnamed: 6'],inplace=True)
pcs_gc_data

,Rnk,Prev.,Diff.,Rider,Team,Pnts
0,1,2.0,1,roglic primoz,Team Jumbo-Visma,1340
1,2,9.0,7,bernal egan,Team INEOS,1196
2,3,3.0,0,valverde alejandro,Movistar Team,960
3,4,6.0,2,quintana nairo,Movistar Team,896
4,5,19.0,14,buchmann emanuel,BORA - hansgrohe,848
...,...,...,...,...,...,...
992,993,972.0,-21,cully jan andrej,Dukla Banska Bystrica,12
993,994,221.0,-773,schillinger andreas,BORA - hansgrohe,12
994,995,708.0,-287,baron felix alejandro,Team Illuminate,12
996,997,801.0,-196,sleen torjus,Uno-X Norwegian Development Team,12


In [153]:
riders_profile = riders_profile.merge(keep_tdf_riders(pcs_gc_data,riders_profile),how='left',on='Rider',suffixes=('','_pcs_gc'))

In [154]:
riders_profile = riders_profile.rename(columns={'Rnk':'Rnk_pcs_gc','Diff.':'Diff._pcs_gc','Pnts':'Pnts_pcs_gc'})
riders_profile.drop(columns=['Prev.','Team_pcs_gc'],inplace=True)

In [160]:
riders_profile.head(10)

,Rider,Age,Team,Rnk_pcs_ind,Diff._pcs_ind,Pnts_pcs_ind,Rnk_pcs_climb,Diff._pcs_climb,Pnts_pcs_climb,Rnk_pcs_sprint,Diff._pcs_sprint,Pnts_pcs_sprint,Rnk_pcs_gc,Diff._pcs_gc,Pnts_pcs_gc
0,kristoff alexander,33,UAE-Team Emirates,28.0,-20,611.0,NaN,NaN,NaN,17.0,-6,335.5,111.0,190,200.0
1,pedersen mads,24,Trek - Segafredo,26.0,72,625.0,396.0,128,10.0,30.0,94,270.5,970.0,-307,13.0
2,bol cees,25,Team Sunweb,141.0,-39,237.0,687.0,-365,2.0,37.0,18,227.5,246.0,432,101.0
3,bennett sam,29,Deceuninck - Quick Step,12.0,6,828.0,128.0,301,58.0,2.0,2,736.5,582.0,49,36.0
4,sagan peter,30,BORA - hansgrohe,19.0,-6,684.0,116.0,-73,68.0,8.0,1,428.0,524.0,-138,43.0
5,viviani elia,31,"Cofidis, Solutions Crédits",41.0,-32,521.0,219.0,-76,25.0,29.0,-24,272.0,525.0,-425,43.0
6,nizzolo giacomo,31,NTT Pro Cycling,13.0,53,807.0,339.0,8,13.0,5.0,18,535.5,NaN,NaN,NaN
7,coquard bryan,28,B&B Hotels - Vital Concept p/b KTM,163.0,-108,196.0,NaN,NaN,NaN,26.0,-20,278.0,161.0,137,151.0
8,turgis anthony,26,Team Total Direct Energie,126.0,-44,250.0,172.0,77,35.0,65.0,54,170.0,109.0,422,200.0
9,stuyven jasper,28,Trek - Segafredo,11.0,15,843.0,91.0,-4,94.0,14.0,24,341.0,124.0,93,185.0


# PCS TT ranking:
Summation of PCS points over a 12-month + 2 weeks overlap period. Races are counted once. After the finish of a stage, the corresponding stage of the previous edition is excluded. 

In [161]:
for page in np.arange(0,1000,200):
    url = f'https://www.procyclingstats.com/rankings.php?id=60938&nation=&team=&page={page}&prev_id=prev&younger=&older=&limit=200&filter=Filter&morefilters='
    pcs_url = requests.get(url)
    print(page,pcs_url)
    body = pcs_url.text
    pcs_tt_data_n = pd.read_html(body)
    if page>0:
        pcs_tt_data = pcs_tt_data.append(pd.DataFrame(pcs_tt_data_n[0]),ignore_index=True)
    else:
        pcs_tt_data = pd.DataFrame(pcs_tt_data_n[0])

0 <Response [200]>
200 <Response [200]>
400 <Response [200]>
600 <Response [200]>
800 <Response [200]>


In [162]:
pcs_tt_data.dropna(inplace=True)
pcs_tt_data['Diff.'] = pcs_tt_data['Diff.'].apply(lambda x: x.replace(chr(9650),''))
pcs_tt_data['Diff.'] = pcs_tt_data['Diff.'].apply(lambda x: x.replace('-','0'))
pcs_tt_data['Diff.'] = pcs_tt_data['Diff.'].apply(lambda x: x.replace(chr(9660),'-'))
pcs_tt_data.Rider = pcs_tt_data.Rider.apply(lambda x: unidecode.unidecode(x.lower()))
pcs_tt_data.drop(columns=['Unnamed: 6'],inplace=True)
pcs_tt_data

,Rnk,Prev.,Diff.,Rider,Team,Pnts
0,1,3.0,2,ganna filippo,INEOS Grenadiers,430
1,2,5.0,3,dennis rohan,INEOS Grenadiers,385
2,3,7.0,4,evenepoel remco,Deceuninck - Quick Step,340
3,4,2.0,-2,campenaerts victor,NTT Pro Cycling,281
4,5,4.0,-1,kung stefan,Groupama - FDJ,256
...,...,...,...,...,...,...
965,966,956.0,-10,goncalves gaspar,Miranda - Mortágua,2
975,976,150.0,-826,nazaret magno,Sindicato Empleados Públicos of San Juan,2
977,978,277.0,-701,arndt nikias,Team Sunweb,2
984,985,701.0,-284,berrade urko,Equipo Kern Pharma,2


In [163]:
riders_profile = riders_profile.merge(keep_tdf_riders(pcs_tt_data,riders_profile),how='left',on='Rider',suffixes=('','_pcs_tt'))

In [164]:
riders_profile = riders_profile.rename(columns={'Rnk':'Rnk_pcs_tt','Diff.':'Diff._pcs_tt','Pnts':'Pnts_pcs_tt'})
riders_profile.drop(columns=['Prev.','Team_pcs_tt'],inplace=True)

In [165]:
riders_profile

,Rider,Age,Team,Rnk_pcs_ind,Diff._pcs_ind,Pnts_pcs_ind,Rnk_pcs_climb,Diff._pcs_climb,Pnts_pcs_climb,Rnk_pcs_sprint,Diff._pcs_sprint,Pnts_pcs_sprint,Rnk_pcs_gc,Diff._pcs_gc,Pnts_pcs_gc,Rnk_pcs_tt,Diff._pcs_tt,Pnts_pcs_tt
0,kristoff alexander,33,UAE-Team Emirates,28.0,-20,611.0,NaN,NaN,NaN,17.0,-6,335.5,111.0,190,200.0,NaN,NaN,NaN
1,pedersen mads,24,Trek - Segafredo,26.0,72,625.0,396.0,128,10.0,30.0,94,270.5,970.0,-307,13.0,NaN,NaN,NaN
2,bol cees,25,Team Sunweb,141.0,-39,237.0,687.0,-365,2.0,37.0,18,227.5,246.0,432,101.0,462.0,-97,10.0
3,bennett sam,29,Deceuninck - Quick Step,12.0,6,828.0,128.0,301,58.0,2.0,2,736.5,582.0,49,36.0,NaN,NaN,NaN
4,sagan peter,30,BORA - hansgrohe,19.0,-6,684.0,116.0,-73,68.0,8.0,1,428.0,524.0,-138,43.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,sivakov pavel,23,INEOS Grenadiers,57.0,-19,431.0,45.0,5,225.0,NaN,NaN,NaN,10.0,247,687.0,NaN,NaN,NaN
172,cras steff,24,Lotto Soudal,502.0,97,58.0,225.0,169,24.0,NaN,NaN,NaN,295.0,-58,84.0,NaN,NaN,NaN
173,ledanois kevin,27,Team Arkéa Samsic,420.0,148,75.0,NaN,NaN,NaN,343.0,86,43.0,607.0,-20,35.0,NaN,NaN,NaN
174,valls rafael,33,Bahrain - McLaren,807.0,-143,24.0,354.0,151,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# PCS team ranking:
Summation of PCS points over a 12-month + 2 weeks overlap period. Races are counted once. After the finish of a stage, the corresponding stage of the previous edition is excluded. 

In [167]:
url = f'https://www.procyclingstats.com/rankings.php?id=50929&nation=&class=&prev_entry_id=50929&filter=Filter&morefilters='
pcs_url = requests.get(url)
print(page,pcs_url)
body = pcs_url.text
pcs_team_data_n = pd.read_html(body)
pcs_team_data = pd.DataFrame(pcs_team_data_n[0])
pcs_team_data

800 <Response [200]>


,Rnk,Prev.,Diff.,Team,Class,Pnt
0,1,1,-,Deceuninck - Quick Step,WT,9229
1,2,2,-,Team Jumbo-Visma,WT,8226
2,3,3,-,BORA - hansgrohe,WT,8117
3,4,4,-,Team INEOS,WT,7757
4,5,5,-,Astana Pro Team,WT,7597
...,...,...,...,...,...,...
239,240,240,-,Gapyeong Cycling Team,CT,1
240,241,241,-,BetPlay Cycling Team,CT,1
241,242,242,-,Pro Touch Wanderers,CLUB,1
242,243,243,-,Orgullo Paisa,CT,1


In [172]:
riders_profile = riders_profile.merge(pcs_team_data,how='left',on='Team',suffixes=('','_pcs_team'))

In [173]:
riders_profile = riders_profile.rename(columns={'Rnk':'Rnk_pcs_team','Diff.':'Diff._pcs_team','Pnt':'Pnts_pcs_team','Class':'Team_class'})
riders_profile.drop(columns=['Prev.'],inplace=True)

In [174]:
riders_profile

,Rider,Age,Team,Rnk_pcs_ind,Diff._pcs_ind,Pnts_pcs_ind,Rnk_pcs_climb,Diff._pcs_climb,Pnts_pcs_climb,Rnk_pcs_sprint,...,Rnk_pcs_gc,Diff._pcs_gc,Pnts_pcs_gc,Rnk_pcs_tt,Diff._pcs_tt,Pnts_pcs_tt,Rnk_pcs_team,Diff._pcs_team,Team_class,Pnts_pcs_team
0,kristoff alexander,33,UAE-Team Emirates,28.0,-20,611.0,NaN,NaN,NaN,17.0,...,111.0,190,200.0,NaN,NaN,NaN,6.0,-,WT,6932.0
1,pedersen mads,24,Trek - Segafredo,26.0,72,625.0,396.0,128,10.0,30.0,...,970.0,-307,13.0,NaN,NaN,NaN,11.0,-,WT,4949.0
2,bol cees,25,Team Sunweb,141.0,-39,237.0,687.0,-365,2.0,37.0,...,246.0,432,101.0,462.0,-97,10.0,15.0,-,WT,4510.0
3,bennett sam,29,Deceuninck - Quick Step,12.0,6,828.0,128.0,301,58.0,2.0,...,582.0,49,36.0,NaN,NaN,NaN,1.0,-,WT,9229.0
4,sagan peter,30,BORA - hansgrohe,19.0,-6,684.0,116.0,-73,68.0,8.0,...,524.0,-138,43.0,NaN,NaN,NaN,3.0,-,WT,8117.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,sivakov pavel,23,INEOS Grenadiers,57.0,-19,431.0,45.0,5,225.0,NaN,...,10.0,247,687.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172,cras steff,24,Lotto Soudal,502.0,97,58.0,225.0,169,24.0,NaN,...,295.0,-58,84.0,NaN,NaN,NaN,8.0,-,WT,6013.0
173,ledanois kevin,27,Team Arkéa Samsic,420.0,148,75.0,NaN,NaN,NaN,343.0,...,607.0,-20,35.0,NaN,NaN,NaN,25.0,-,PCT,2229.0
174,valls rafael,33,Bahrain - McLaren,807.0,-143,24.0,354.0,151,12.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [207]:
riders_profile[riders_profile.Team_class.isna()]['Team'].unique()

array([], dtype=object)

In [205]:
pcs_team_data[pcs_team_data.Team == 'Israel Cycling Academy']

,Rnk,Prev.,Diff.,Team,Class,Pnt
15,16,16,-,Israel Cycling Academy,PCT,3659


In [206]:
idx = riders_profile[riders_profile.Team == 'Israel Start-Up Nation'].index
riders_profile.loc[idx,['Rnk_pcs_team','Diff._pcs_team','Team_class','Pnts_pcs_team']] = [16,'-','PCT',3659]

In [208]:
riders_profile

,Rider,Age,Team,Rnk_pcs_ind,Diff._pcs_ind,Pnts_pcs_ind,Rnk_pcs_climb,Diff._pcs_climb,Pnts_pcs_climb,Rnk_pcs_sprint,...,Rnk_pcs_gc,Diff._pcs_gc,Pnts_pcs_gc,Rnk_pcs_tt,Diff._pcs_tt,Pnts_pcs_tt,Rnk_pcs_team,Diff._pcs_team,Team_class,Pnts_pcs_team
0,kristoff alexander,33,UAE-Team Emirates,28.0,-20,611.0,NaN,NaN,NaN,17.0,...,111.0,190,200.0,NaN,NaN,NaN,6.0,-,WT,6932.0
1,pedersen mads,24,Trek - Segafredo,26.0,72,625.0,396.0,128,10.0,30.0,...,970.0,-307,13.0,NaN,NaN,NaN,11.0,-,WT,4949.0
2,bol cees,25,Team Sunweb,141.0,-39,237.0,687.0,-365,2.0,37.0,...,246.0,432,101.0,462.0,-97,10.0,15.0,-,WT,4510.0
3,bennett sam,29,Deceuninck - Quick Step,12.0,6,828.0,128.0,301,58.0,2.0,...,582.0,49,36.0,NaN,NaN,NaN,1.0,-,WT,9229.0
4,sagan peter,30,BORA - hansgrohe,19.0,-6,684.0,116.0,-73,68.0,8.0,...,524.0,-138,43.0,NaN,NaN,NaN,3.0,-,WT,8117.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,sivakov pavel,23,INEOS Grenadiers,57.0,-19,431.0,45.0,5,225.0,NaN,...,10.0,247,687.0,NaN,NaN,NaN,4.0,-,WT,7757.0
172,cras steff,24,Lotto Soudal,502.0,97,58.0,225.0,169,24.0,NaN,...,295.0,-58,84.0,NaN,NaN,NaN,8.0,-,WT,6013.0
173,ledanois kevin,27,Team Arkéa Samsic,420.0,148,75.0,NaN,NaN,NaN,343.0,...,607.0,-20,35.0,NaN,NaN,NaN,25.0,-,PCT,2229.0
174,valls rafael,33,Bahrain - McLaren,807.0,-143,24.0,354.0,151,12.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,-,WT,4948.0


### SAVED DATA WITH RANK,CLIMB,SPRINT FROM PCS + GC,TT and TEAM

In [370]:
riders_profile.to_csv('riders_profile.csv')

In [342]:
#riders_profile.drop(columns=['Rnk_uci_ind','Pnts_uci_ind','Rnk_uci_gc','Pnts_uci_gc','Rnk_uci_team','Pnts_uci_team']).to_csv('riders_profile.csv')

In [366]:
teams = pd.DataFrame(riders_profile.Team.unique(),columns=['Team'])
teams

,Team
0,UAE-Team Emirates
1,Trek - Segafredo
2,Team Sunweb
3,Deceuninck - Quick Step
4,BORA - hansgrohe
5,"Cofidis, Solutions Crédits"
6,NTT Pro Cycling
7,B&B Hotels - Vital Concept p/b KTM
8,Team Total Direct Energie
9,AG2R La Mondiale


In [367]:
teams['Team Code'] = ['UAD','TFS','SUN','DQT','BOH','COF','NTT','BVC','TDE','ALM','CCC','MTS','AST','EF1',
'ARK','LTS','TBM','INS','TJV','ISN','MOV','GFC']
teams

,Team,Team Code
0,UAE-Team Emirates,UAD
1,Trek - Segafredo,TFS
2,Team Sunweb,SUN
3,Deceuninck - Quick Step,DQT
4,BORA - hansgrohe,BOH
5,"Cofidis, Solutions Crédits",COF
6,NTT Pro Cycling,NTT
7,B&B Hotels - Vital Concept p/b KTM,BVC
8,Team Total Direct Energie,TDE
9,AG2R La Mondiale,ALM


In [369]:
riders_profile = riders_profile.merge(teams,on=['Team'],how='left')

In [387]:
riders_profile = pd.read_csv('riders_profile.csv',index_col=0)
riders_profile  

,Rider,Age,Team,Rnk_pcs_ind,Diff._pcs_ind,Pnts_pcs_ind,Rnk_pcs_climb,Diff._pcs_climb,Pnts_pcs_climb,Rnk_pcs_sprint,...,Diff._pcs_gc,Pnts_pcs_gc,Rnk_pcs_tt,Diff._pcs_tt,Pnts_pcs_tt,Rnk_pcs_team,Diff._pcs_team,Team_class,Pnts_pcs_team,Team Code
0,kristoff alexander,33,UAE-Team Emirates,28.0,-20.0,611.0,724.0,0.0,0.0,17.0,...,190.0,200.0,979.0,0.0,0.0,6.0,-,WT,6932.0,UAD
1,pedersen mads,24,Trek - Segafredo,26.0,72.0,625.0,396.0,128.0,10.0,30.0,...,-307.0,13.0,979.0,0.0,0.0,11.0,-,WT,4949.0,TFS
2,bol cees,25,Team Sunweb,141.0,-39.0,237.0,687.0,-365.0,2.0,37.0,...,432.0,101.0,462.0,-97.0,10.0,15.0,-,WT,4510.0,SUN
3,bennett sam,29,Deceuninck - Quick Step,12.0,6.0,828.0,128.0,301.0,58.0,2.0,...,49.0,36.0,979.0,0.0,0.0,1.0,-,WT,9229.0,DQT
4,sagan peter,30,BORA - hansgrohe,19.0,-6.0,684.0,116.0,-73.0,68.0,8.0,...,-138.0,43.0,979.0,0.0,0.0,3.0,-,WT,8117.0,BOH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,sivakov pavel,23,INEOS Grenadiers,57.0,-19.0,431.0,45.0,5.0,225.0,940.0,...,247.0,687.0,979.0,0.0,0.0,4.0,-,WT,7757.0,INS
172,cras steff,24,Lotto Soudal,502.0,97.0,58.0,225.0,169.0,24.0,940.0,...,-58.0,84.0,979.0,0.0,0.0,8.0,-,WT,6013.0,LTS
173,ledanois kevin,27,Team Arkéa Samsic,420.0,148.0,75.0,724.0,0.0,0.0,343.0,...,-20.0,35.0,979.0,0.0,0.0,25.0,-,PCT,2229.0,ARK
174,valls rafael,33,Bahrain - McLaren,807.0,-143.0,24.0,354.0,151.0,12.0,940.0,...,0.0,0.0,979.0,0.0,0.0,12.0,-,WT,4948.0,TBM


In [388]:
riders_profile['last_name'] = riders_profile.Rider.apply(lambda x: x.split()[0])

In [389]:
uci_ind_data = pd.read_excel('UCIIndividualRanking.xlsx')
uci_ind_data

,Rank,UCI ID,Name,Nationality,Team Code,Age,Points
0,1,10008888921,ROGLIČ Primož,SLOVENIA,TJV,30,3343.57
1,2,10003401246,FUGLSANG Jakob,DENMARK,AST,35,3106.50
2,3,10007155752,ALAPHILIPPE Julian,FRANCE,DQT,28,3028.95
3,4,10005467952,ULISSI Diego,ITALY,UAD,31,2246.00
4,5,10004451371,VAN AVERMAET Greg,BELGIUM,CCC,35,2160.00
...,...,...,...,...,...,...,...
2564,2565,10008659454,ABRAHAMSEN Jonas,NORWAY,UXT,25,0.43
2565,2565,10008705025,EVTUSHENKO Alexander,RUSSIAN FEDERATION,STC,27,0.43
2566,2565,10009194772,KULIKOV Vladislav,RUSSIAN FEDERATION,GAZ,24,0.43
2567,2565,10006473318,VOROBYEV Anton,RUSSIAN FEDERATION,NaN,30,0.43


In [390]:
uci_ind_data['Name'] = uci_ind_data.Name.apply(lambda x: unidecode.unidecode(x.lower()))
uci_ind_data['last_name'] = uci_ind_data.Name.apply(lambda x: x.split()[0])

In [391]:
uci_ind_data

,Rank,UCI ID,Name,Nationality,Team Code,Age,Points,last_name
0,1,10008888921,roglic primoz,SLOVENIA,TJV,30,3343.57,roglic
1,2,10003401246,fuglsang jakob,DENMARK,AST,35,3106.50,fuglsang
2,3,10007155752,alaphilippe julian,FRANCE,DQT,28,3028.95,alaphilippe
3,4,10005467952,ulissi diego,ITALY,UAD,31,2246.00,ulissi
4,5,10004451371,van avermaet greg,BELGIUM,CCC,35,2160.00,van
...,...,...,...,...,...,...,...,...
2564,2565,10008659454,abrahamsen jonas,NORWAY,UXT,25,0.43,abrahamsen
2565,2565,10008705025,evtushenko alexander,RUSSIAN FEDERATION,STC,27,0.43,evtushenko
2566,2565,10009194772,kulikov vladislav,RUSSIAN FEDERATION,GAZ,24,0.43,kulikov
2567,2565,10006473318,vorobyev anton,RUSSIAN FEDERATION,NaN,30,0.43,vorobyev


In [392]:
uci_ind_data = uci_ind_data.rename(columns={'Name':'Rider'})

In [393]:
#riders_profile = riders_profile.merge(keep_tdf_riders(uci_ind_data,riders_profile),how='left',on='Rider',suffixes=('','_uci_ind'))
riders_profile = riders_profile.merge(uci_ind_data,how='left',on=['Team Code','Age','last_name'],suffixes=('','_uci_ind'))

In [394]:
riders_profile[riders_profile.Rank.isna()]

,Rider,Age,Team,Rnk_pcs_ind,Diff._pcs_ind,Pnts_pcs_ind,Rnk_pcs_climb,Diff._pcs_climb,Pnts_pcs_climb,Rnk_pcs_sprint,...,Diff._pcs_team,Team_class,Pnts_pcs_team,Team Code,last_name,Rank,UCI ID,Rider_uci_ind,Nationality,Points
54,grellier fabien,25,Team Total Direct Energie,1459.0,0.0,0.0,724.0,0.0,0.0,940.0,...,-,PCT,3497.0,TDE,grellier,NaN,NaN,NaN,NaN,NaN
55,reza kevin,32,B&B Hotels - Vital Concept p/b KTM,1459.0,0.0,0.0,724.0,0.0,0.0,940.0,...,-,PCT,2201.0,BVC,reza,NaN,NaN,NaN,NaN,NaN
132,izagirre ion,31,Astana Pro Team,65.0,-16.0,379.0,101.0,-68.0,78.0,940.0,...,-,WT,7597.0,AST,izagirre,NaN,NaN,NaN,NaN,NaN
160,bonnet william,38,Groupama - FDJ,1459.0,0.0,0.0,724.0,0.0,0.0,940.0,...,-,WT,4705.0,GFC,bonnet,NaN,NaN,NaN,NaN,NaN


In [397]:
riders_profile.Rnk_pcs_ind.value_counts().sort_values()

337.0     1
240.0     1
174.0     1
147.0     1
420.0     1
         ..
141.0     1
11.0      1
341.0     2
44.0      2
1459.0    4
Name: Rnk_pcs_ind, Length: 173, dtype: int64

In [400]:
riders_profile[riders_profile['Rnk_pcs_ind']==341]

,Rider,Age,Team,Rnk_pcs_ind,Diff._pcs_ind,Pnts_pcs_ind,Rnk_pcs_climb,Diff._pcs_climb,Pnts_pcs_climb,Rnk_pcs_sprint,...,Diff._pcs_team,Team_class,Pnts_pcs_team,Team Code,last_name,Rank,UCI ID,Rider_uci_ind,Nationality,Points
103,de marchi alessandro,34,CCC Team,341.0,61.0,98.0,104.0,57.0,76.0,940.0,...,-,WT,3199.0,CCC,de,139.0,1.000309e+10,de marchi alessandro,ITALY,377.0
104,de marchi alessandro,34,CCC Team,341.0,61.0,98.0,104.0,57.0,76.0,940.0,...,-,WT,3199.0,CCC,de,218.0,1.000745e+10,de la parte victor,SPAIN,220.0


In [402]:
# rows to drop 117,104
riders_profile.drop(index=[104,117],inplace=True)

## Saved riders_profile with UCI ID

In [403]:
riders_profile.to_csv('riders_profile.csv')

In [406]:
riders_profile.drop(columns=['Rider_uci_ind','Nationality'],inplace=True)

In [407]:
riders_profile = riders_profile.rename(columns={'Rank':'Rnk_uci_ind','Points':'Pnts_uci_ind'})

In [408]:
riders_profile

,Rider,Age,Team,Rnk_pcs_ind,Diff._pcs_ind,Pnts_pcs_ind,Rnk_pcs_climb,Diff._pcs_climb,Pnts_pcs_climb,Rnk_pcs_sprint,...,Pnts_pcs_tt,Rnk_pcs_team,Diff._pcs_team,Team_class,Pnts_pcs_team,Team Code,last_name,Rnk_uci_ind,UCI ID,Pnts_uci_ind
0,kristoff alexander,33,UAE-Team Emirates,28.0,-20.0,611.0,724.0,0.0,0.0,17.0,...,0.0,6.0,-,WT,6932.0,UAD,kristoff,14.0,1.000327e+10,1766.00
1,pedersen mads,24,Trek - Segafredo,26.0,72.0,625.0,396.0,128.0,10.0,30.0,...,0.0,11.0,-,WT,4949.0,TFS,pedersen,50.0,1.000869e+10,958.00
2,bol cees,25,Team Sunweb,141.0,-39.0,237.0,687.0,-365.0,2.0,37.0,...,10.0,15.0,-,WT,4510.0,SUN,bol,216.0,1.000940e+10,222.88
3,bennett sam,29,Deceuninck - Quick Step,12.0,6.0,828.0,128.0,301.0,58.0,2.0,...,0.0,1.0,-,WT,9229.0,DQT,bennett,51.0,1.000660e+10,947.50
4,sagan peter,30,BORA - hansgrohe,19.0,-6.0,684.0,116.0,-73.0,68.0,8.0,...,0.0,3.0,-,WT,8117.0,BOH,sagan,8.0,1.000546e+10,1978.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,sivakov pavel,23,INEOS Grenadiers,57.0,-19.0,431.0,45.0,5.0,225.0,940.0,...,0.0,4.0,-,WT,7757.0,INS,sivakov,67.0,1.001113e+10,725.00
174,cras steff,24,Lotto Soudal,502.0,97.0,58.0,225.0,169.0,24.0,940.0,...,0.0,8.0,-,WT,6013.0,LTS,cras,774.0,1.001009e+10,54.00
175,ledanois kevin,27,Team Arkéa Samsic,420.0,148.0,75.0,724.0,0.0,0.0,343.0,...,0.0,25.0,-,PCT,2229.0,ARK,ledanois,420.0,1.000759e+10,118.00
176,valls rafael,33,Bahrain - McLaren,807.0,-143.0,24.0,354.0,151.0,12.0,940.0,...,0.0,12.0,-,WT,4948.0,TBM,valls,311.0,1.000463e+10,160.00


In [412]:
no_id = riders_profile[riders_profile['UCI ID'].isna()].index
for i in range(len(no_id)):
    riders_profile.loc[no_id[i],'UCI ID'] = i

In [413]:
riders_profile.loc[no_id]

,Rider,Age,Team,Rnk_pcs_ind,Diff._pcs_ind,Pnts_pcs_ind,Rnk_pcs_climb,Diff._pcs_climb,Pnts_pcs_climb,Rnk_pcs_sprint,...,Pnts_pcs_tt,Rnk_pcs_team,Diff._pcs_team,Team_class,Pnts_pcs_team,Team Code,last_name,Rnk_uci_ind,UCI ID,Pnts_uci_ind
54,grellier fabien,25,Team Total Direct Energie,1459.0,0.0,0.0,724.0,0.0,0.0,940.0,...,0.0,19.0,-,PCT,3497.0,TDE,grellier,NaN,0.0,NaN
55,reza kevin,32,B&B Hotels - Vital Concept p/b KTM,1459.0,0.0,0.0,724.0,0.0,0.0,940.0,...,0.0,26.0,-,PCT,2201.0,BVC,reza,NaN,1.0,NaN
132,izagirre ion,31,Astana Pro Team,65.0,-16.0,379.0,101.0,-68.0,78.0,940.0,...,42.0,5.0,-,WT,7597.0,AST,izagirre,NaN,2.0,NaN
160,bonnet william,38,Groupama - FDJ,1459.0,0.0,0.0,724.0,0.0,0.0,940.0,...,0.0,14.0,-,WT,4705.0,GFC,bonnet,NaN,3.0,NaN


In [414]:
uci_gc_data = pd.read_excel('UCIStageRace.xlsx')
uci_gc_data

,Rank,UCI ID,Name,Nationality,Team Code,Age,Points
0,1,10008888921,ROGLIČ Primož,SLOVENIA,TJV,31,3113.57
1,2,10014972740,POGAČAR Tadej,SLOVENIA,UAD,22,1755.00
2,3,10006738854,QUINTANA Nairo,COLOMBIA,ARK,30,1691.00
3,4,10007155752,ALAPHILIPPE Julian,FRANCE,DQT,28,1476.95
4,5,10009584489,BERNAL GOMEZ Egan Arley,COLOMBIA,INS,23,1446.75
...,...,...,...,...,...,...,...
1394,1392,10008659454,ABRAHAMSEN Jonas,NORWAY,UXT,25,0.43
1395,1392,10008705025,EVTUSHENKO Alexander,RUSSIAN FEDERATION,STC,27,0.43
1396,1392,10009194772,KULIKOV Vladislav,RUSSIAN FEDERATION,GAZ,24,0.43
1397,1392,10015221102,JOHANSEN Julius,DENMARK,UXT,21,0.43


In [415]:
uci_gc_data['Name'] = uci_gc_data.Name.apply(lambda x: unidecode.unidecode(x.lower()))
uci_gc_data = uci_gc_data.rename(columns={'Name':'Rider'})
riders_profile = riders_profile.merge(uci_gc_data,how='left',on='UCI ID',suffixes=('','_uci_gc'))

In [417]:
riders_profile.drop(columns=['Rider_uci_gc','Nationality','Team Code_uci_gc','Age_uci_gc'],inplace=True)
riders_profile = riders_profile.rename(columns={'Rank':'Rnk_uci_gc','Points':'Pnts_uci_gc'})
riders_profile

,Rider,Age,Team,Rnk_pcs_ind,Diff._pcs_ind,Pnts_pcs_ind,Rnk_pcs_climb,Diff._pcs_climb,Pnts_pcs_climb,Rnk_pcs_sprint,...,Team_class,Pnts_pcs_team,Team Code,last_name,Rnk_uci_ind,UCI ID,Pnts_uci_ind,Rnk_uci_gc,Rider_uci_gc,Pnts_uci_gc
0,kristoff alexander,33,UAE-Team Emirates,28.0,-20.0,611.0,724.0,0.0,0.0,17.0,...,WT,6932.0,UAD,kristoff,14.0,1.000327e+10,1766.00,181.0,kristoff alexander,128.00
1,pedersen mads,24,Trek - Segafredo,26.0,72.0,625.0,396.0,128.0,10.0,30.0,...,WT,4949.0,TFS,pedersen,50.0,1.000869e+10,958.00,280.0,pedersen mads,76.00
2,bol cees,25,Team Sunweb,141.0,-39.0,237.0,687.0,-365.0,2.0,37.0,...,WT,4510.0,SUN,bol,216.0,1.000940e+10,222.88,448.0,bol cees,41.88
3,bennett sam,29,Deceuninck - Quick Step,12.0,6.0,828.0,128.0,301.0,58.0,2.0,...,WT,9229.0,DQT,bennett,51.0,1.000660e+10,947.50,25.0,bennett sam,709.50
4,sagan peter,30,BORA - hansgrohe,19.0,-6.0,684.0,116.0,-73.0,68.0,8.0,...,WT,8117.0,BOH,sagan,8.0,1.000546e+10,1978.00,44.0,sagan peter,445.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,sivakov pavel,23,INEOS Grenadiers,57.0,-19.0,431.0,45.0,5.0,225.0,940.0,...,WT,7757.0,INS,sivakov,67.0,1.001113e+10,725.00,40.0,sivakov pavel,475.00
172,cras steff,24,Lotto Soudal,502.0,97.0,58.0,225.0,169.0,24.0,940.0,...,WT,6013.0,LTS,cras,774.0,1.001009e+10,54.00,453.0,cras steff,41.00
173,ledanois kevin,27,Team Arkéa Samsic,420.0,148.0,75.0,724.0,0.0,0.0,343.0,...,PCT,2229.0,ARK,ledanois,420.0,1.000759e+10,118.00,328.0,ledanois kevin,63.00
174,valls rafael,33,Bahrain - McLaren,807.0,-143.0,24.0,354.0,151.0,12.0,940.0,...,WT,4948.0,TBM,valls,311.0,1.000463e+10,160.00,495.0,valls ferri rafael,35.00


In [419]:
uci_team_data = pd.read_excel('UCITeamRanking.xlsx')
uci_team_data

,Rank,Name,Team Code,Nationality,Points
0,1,DECEUNINCK - QUICK - STEP,DQT,BELGIUM,4333.83
1,2,ASTANA PRO TEAM,AST,KAZAKHSTAN,4326.00
2,3,UAE TEAM EMIRATES,UAD,UNITED ARAB EMIRATES,4180.00
3,4,BORA - HANSGROHE,BOH,GERMANY,3936.00
4,5,TREK - SEGAFREDO,TFS,UNITED STATES OF AMERICA,3847.00
...,...,...,...,...,...
167,168,EQUIPO CONTINENTAL SAN LUIS,CSL,ARGENTINA,3.00
168,169,SANGEMINI TREVIGIANI MG.K VIS,SAT,ITALY,3.00
169,170,Team COLOQUICK,TCQ,DENMARK,1.00
170,171,MIRANDA - MORTÁGUA,MIR,PORTUGAL,1.00


In [420]:
riders_profile = riders_profile.merge(uci_team_data,how='left',on='Team Code',suffixes=('','_uci_team'))
riders_profile

,Rider,Age,Team,Rnk_pcs_ind,Diff._pcs_ind,Pnts_pcs_ind,Rnk_pcs_climb,Diff._pcs_climb,Pnts_pcs_climb,Rnk_pcs_sprint,...,last_name,Rnk_uci_ind,UCI ID,Pnts_uci_ind,Rnk_uci_gc,Pnts_uci_gc,Rank,Name,Nationality,Points
0,kristoff alexander,33,UAE-Team Emirates,28.0,-20.0,611.0,724.0,0.0,0.0,17.0,...,kristoff,14.0,1.000327e+10,1766.00,181.0,128.00,3,UAE TEAM EMIRATES,UNITED ARAB EMIRATES,4180.00
1,pedersen mads,24,Trek - Segafredo,26.0,72.0,625.0,396.0,128.0,10.0,30.0,...,pedersen,50.0,1.000869e+10,958.00,280.0,76.00,5,TREK - SEGAFREDO,UNITED STATES OF AMERICA,3847.00
2,bol cees,25,Team Sunweb,141.0,-39.0,237.0,687.0,-365.0,2.0,37.0,...,bol,216.0,1.000940e+10,222.88,448.0,41.88,11,TEAM SUNWEB,GERMANY,2409.42
3,bennett sam,29,Deceuninck - Quick Step,12.0,6.0,828.0,128.0,301.0,58.0,2.0,...,bennett,51.0,1.000660e+10,947.50,25.0,709.50,1,DECEUNINCK - QUICK - STEP,BELGIUM,4333.83
4,sagan peter,30,BORA - hansgrohe,19.0,-6.0,684.0,116.0,-73.0,68.0,8.0,...,sagan,8.0,1.000546e+10,1978.00,44.0,445.00,4,BORA - HANSGROHE,GERMANY,3936.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,sivakov pavel,23,INEOS Grenadiers,57.0,-19.0,431.0,45.0,5.0,225.0,940.0,...,sivakov,67.0,1.001113e+10,725.00,40.0,475.00,10,TEAM INEOS,GREAT BRITAIN,2591.00
172,cras steff,24,Lotto Soudal,502.0,97.0,58.0,225.0,169.0,24.0,940.0,...,cras,774.0,1.001009e+10,54.00,453.0,41.00,19,LOTTO SOUDAL,BELGIUM,1359.00
173,ledanois kevin,27,Team Arkéa Samsic,420.0,148.0,75.0,724.0,0.0,0.0,343.0,...,ledanois,420.0,1.000759e+10,118.00,328.0,63.00,12,TEAM ARKEA - SAMSIC,FRANCE,2214.00
174,valls rafael,33,Bahrain - McLaren,807.0,-143.0,24.0,354.0,151.0,12.0,940.0,...,valls,311.0,1.000463e+10,160.00,495.0,35.00,16,BAHRAIN - MCLAREN,BAHRAIN,1799.00


In [421]:
riders_profile.drop(columns=['last_name','Name','Nationality'],inplace=True)
riders_profile = riders_profile.rename(columns={'Rank':'Rnk_uci_team','Points':'Pnts_uci_team'})

In [422]:
riders_profile

,Rider,Age,Team,Rnk_pcs_ind,Diff._pcs_ind,Pnts_pcs_ind,Rnk_pcs_climb,Diff._pcs_climb,Pnts_pcs_climb,Rnk_pcs_sprint,...,Pnts_pcs_team,Team Code,last_name,Rnk_uci_ind,UCI ID,Pnts_uci_ind,Rnk_uci_gc,Pnts_uci_gc,Rnk_uci_team,Pnts_uci_team
0,kristoff alexander,33,UAE-Team Emirates,28.0,-20.0,611.0,724.0,0.0,0.0,17.0,...,6932.0,UAD,kristoff,14.0,1.000327e+10,1766.00,181.0,128.00,3,4180.00
1,pedersen mads,24,Trek - Segafredo,26.0,72.0,625.0,396.0,128.0,10.0,30.0,...,4949.0,TFS,pedersen,50.0,1.000869e+10,958.00,280.0,76.00,5,3847.00
2,bol cees,25,Team Sunweb,141.0,-39.0,237.0,687.0,-365.0,2.0,37.0,...,4510.0,SUN,bol,216.0,1.000940e+10,222.88,448.0,41.88,11,2409.42
3,bennett sam,29,Deceuninck - Quick Step,12.0,6.0,828.0,128.0,301.0,58.0,2.0,...,9229.0,DQT,bennett,51.0,1.000660e+10,947.50,25.0,709.50,1,4333.83
4,sagan peter,30,BORA - hansgrohe,19.0,-6.0,684.0,116.0,-73.0,68.0,8.0,...,8117.0,BOH,sagan,8.0,1.000546e+10,1978.00,44.0,445.00,4,3936.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,sivakov pavel,23,INEOS Grenadiers,57.0,-19.0,431.0,45.0,5.0,225.0,940.0,...,7757.0,INS,sivakov,67.0,1.001113e+10,725.00,40.0,475.00,10,2591.00
172,cras steff,24,Lotto Soudal,502.0,97.0,58.0,225.0,169.0,24.0,940.0,...,6013.0,LTS,cras,774.0,1.001009e+10,54.00,453.0,41.00,19,1359.00
173,ledanois kevin,27,Team Arkéa Samsic,420.0,148.0,75.0,724.0,0.0,0.0,343.0,...,2229.0,ARK,ledanois,420.0,1.000759e+10,118.00,328.0,63.00,12,2214.00
174,valls rafael,33,Bahrain - McLaren,807.0,-143.0,24.0,354.0,151.0,12.0,940.0,...,4948.0,TBM,valls,311.0,1.000463e+10,160.00,495.0,35.00,16,1799.00


In [425]:
riders_profile.drop(columns=['Team Code'],inplace=True)

In [426]:
riders_profile.to_csv('riders_profile.csv')